In [1]:
import numpy as np
import torch
import json
import os

from libraries.dataset        import standardize_dataset, check_extend_POSCAR
from libraries.graph          import graph_POSCAR_encoding
from libraries.structure      import compute_diffraction_pattern
from torch_geometric.data     import Data
from pymatgen.core            import Structure

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# In case database is created from scratch (otherwise, it is not being used)
data_path = '../MP/Loaded_MOSES'

# Define diffraction type ('neutron', 'xrd' or 'EPA')
target = 'None'

# Define folder in which all data will be stored
data_folder = f'data/GM_molecules'

# Define name for storing dataset basic description
dataset_parameters_name = f'{data_folder}/dataset_parameters.json'

encoding_type      = 'sphere-images'  # 'voronoi' or 'sphere-images'
distance_threshold = 6  # Used in general

minimum_lattice_vector = 0 * distance_threshold  # Allowing three convolutions

# Define basic dataset parameters for tracking data
dataset_parameters = {
    'input_folder': data_path,
    'output_folder': data_folder,
    'target': target,
    'encoding_type': encoding_type,
    'distance_threshold': distance_threshold,
    'minimum_lattice_vector': minimum_lattice_vector
}

if not os.path.exists(data_folder):
    os.system(f'mkdir {data_folder}')

# Dump the dictionary with numpy arrays to a JSON file
with open(dataset_parameters_name, 'w') as json_file:
    json.dump(dataset_parameters, json_file)

# Generation of graph database for training

Load the datasets, already standardized if possible.

In [ ]:
# Generate the raw dataset from scratch, and standardize it

# Read all materials within the database
materials = os.listdir(data_path)

dataset = []
labels  = []
for material in materials:
    try:
        # Try to read the polymorphs
        polymorphs = os.listdir(f'{data_path}/{material}')
    except:
        continue
    
    #print(material)
    for polymorf in polymorphs:
        # Path to folder containing the POSCAR
        path_to_POSCAR = f'{data_path}/{material}/{polymorf}'
        
        # Check that the folder is valid
        if os.path.exists(path_to_POSCAR):
            #print(f'\t{polymorf}')
            
            try:
                # Load pymatgen structure object
                structure = Structure.from_file(f'{path_to_POSCAR}/POSCAR')
                
                # Check that POSCAR is big enough, otherwise extend it where necessary
                #structure = check_extend_POSCAR(structure, minimum_lattice_vector)
                
                nodes, edges, attributes = graph_POSCAR_encoding(structure,
                                                                 encoding_type=encoding_type,
                                                                 distance_threshold=distance_threshold)
            except:
                print(f'Error: {material} {polymorf} not loaded')
                continue
            
            if target == 'EPA':
                # Load ground state energy per atom
                extracted_target = [float(np.loadtxt(f'{path_to_POSCAR}/EPA'))]
            elif (target == 'neutron') or (target == 'xrd'):
                # Compute diffraction pattern from given structure
                extracted_target = compute_diffraction_pattern(structure, diffraction=target)
            else:
                # Do not extract anything
                extracted_target = [0]
            
            # Construct temporal graph structure
            graph = Data(x=nodes,
                         edge_index=edges.t().contiguous(),
                         edge_attr=attributes.ravel(),
                         y=torch.tensor(extracted_target, dtype=torch.float)
                        )

            # Append to dataset and labels
            dataset.append(graph)
            labels.append(f'{material}-{polymorf}')

Error: 440371 mol not loaded
Error: 236549 mol not loaded
Error: 1002329 mol not loaded
Error: 119473 mol not loaded
Error: 697036 mol not loaded
Error: 769329 mol not loaded
Error: 227167 mol not loaded
Error: 1056589 mol not loaded
Error: 169932 mol not loaded
Error: 197036 mol not loaded
Error: 431172 mol not loaded
Error: 827817 mol not loaded
Error: 736159 mol not loaded
Error: 811470 mol not loaded
Error: 962100 mol not loaded
Error: 215742 mol not loaded
Error: 360878 mol not loaded
Error: 629583 mol not loaded
Error: 263809 mol not loaded
Error: 391104 mol not loaded
Error: 401568 mol not loaded
Error: 916192 mol not loaded
Error: 1048461 mol not loaded
Error: 693050 mol not loaded
Error: 1061380 mol not loaded
Error: 459329 mol not loaded
Error: 327167 mol not loaded
Error: 914318 mol not loaded
Error: 488610 mol not loaded
Error: 62375 mol not loaded
Error: 122332 mol not loaded
Error: 1018832 mol not loaded
Error: 99296 mol not loaded
Error: 523132 mol not loaded
Error: 5296

In [ ]:
len(dataset)

In [ ]:
# Standardize dataset
dataset_std, dataset_parameters = standardize_dataset(dataset)

# Save dataset

In [11]:
labels_name                 = f'{data_folder}/labels.pt'
dataset_name                = f'{data_folder}/dataset.pt'
dataset_name_std            = f'{data_folder}/standardized_dataset.pt'
dataset_parameters_name_std = f'{data_folder}/standardized_parameters.json'  # Parameters for rescaling the predictions

torch.save(labels,      labels_name)
torch.save(dataset,     dataset_name)
torch.save(dataset_std, dataset_name_std)

# Convert torch tensors to numpy arrays
numpy_dict = {key: value.cpu().numpy().tolist() for key, value in dataset_parameters.items()}

# Dump the dictionary with numpy arrays to a JSON file
with open(dataset_parameters_name_std, 'w') as json_file:
    json.dump(numpy_dict, json_file)